# Step 2 - Database Engineering

In [ ]:
# Use SQLAlchemy to model your table schemas and create a sqlite database for your tables. 
# You will need one table for measurements and one for stations.

# Create a Jupyter Notebook called database_engineering.ipynb and use this to complete 
# all of your Database Engineering work.

# Use Pandas to read your cleaned measurements and stations CSV data.
# Use the engine and connection string to create a database called hawaii.sqlite.

# Use declarative_base and create ORM classes for each table.

# You will need a class for Measurement and for Station.
# Make sure to define your primary keys.

# Once you have your ORM classes defined, create the tables in the database using create_all.

### Load cleaned csv's into pandas DataFrames 

In [2]:
# Import dependencies 
import pandas as pd

In [5]:
csv_path = "clean_hawaii_stations.csv"
stations_df = pd.read_csv(csv_path)
stations_df = stations_df.rename(columns={'Unnamed: 0':'id'})
stations_df.head()

,id,station,name,latitude,longitude,elevation
0,0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [6]:
csv_path = "clean_hawaii_measurements.csv"
weather_df = pd.read_csv(csv_path)
weather_df = weather_df.rename(columns={'Unnamed: 0':'id'})
weather_df.head()

,id,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,5,USC00519397,2010-01-07,0.06,70


# Database Creation

In [7]:
# SQLAlchemy
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
from sqlalchemy.orm import Session

In [8]:
# Create an engine to a SQLite database file 
engine = create_engine("sqlite:///hawaii.sqlite")

In [9]:
# Create a connection to the engine
conn = engine.connect()

In [10]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class

Base = declarative_base()

class Measurement (Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)

# Define a Station class
class Station (Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)


In [11]:
# Use `create_all` to create table in database
Base.metadata.create_all(engine)

In [12]:
# Create a list of data to write
data1 = stations_df.to_dict(orient='records')
data2 = weather_df.to_dict(orient='records')

In [13]:
# Check list of dictionaries 
print(data1[:5])
print("------------")
print(data2[:5])

[{'id': 0, 'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'id': 1, 'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'id': 2, 'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'id': 3, 'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'id': 4, 'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}]
------------
[{'id': 0, 'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'id': 1, 'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'id': 2, 'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'id': 3, 'station': 'USC00519397', 'date': '20

In [14]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [15]:
# Save the reference to the `measurements` table as a variable called `table`
table1 = sqlalchemy.Table('stations', metadata, autoload=True)
table2 = sqlalchemy.Table('measurements', metadata, autoload=True)

In [16]:
# Use `table.insert()` to insert data into tables
# The SQL table is populated during this step
conn.execute(table1.insert(), data1)
conn.execute(table2.insert(), data2)

In [17]:
# Check data 
conn.execute("select * from stations limit 5").fetchall()

[(0, 0, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (1, 1, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (2, 2, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (3, 3, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (4, 4, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [18]:
# Check data 
conn.execute("select * from measurements limit 5").fetchall()

[(0, 0, 'USC00519397', '2010-01-01', 0.08, 65),
 (1, 1, 'USC00519397', '2010-01-02', 0.0, 63),
 (2, 2, 'USC00519397', '2010-01-03', 0.0, 74),
 (3, 3, 'USC00519397', '2010-01-04', 0.0, 76),
 (4, 5, 'USC00519397', '2010-01-07', 0.06, 70)]

### Database engineering is complete 